### The code and data are adapted from:  https://medium.com/@vitalshchutski/french-nlp-entamez-le-camembert-avec-les-librairies-fast-bert-et-transformers-14e65f84c148

In [34]:
# -*- coding: utf-8 -*-
# !conda install torch
# !pip install fast-bert==1.9.1
# !mkdir model
# !mkdir finetuned_model

In [1]:
#!pip install win_unicode_console

In [32]:
! chcp 65001

Page de codes activeÿ: 65001


In [1]:
raise Exception(u'щ')

Exception: щ

In [ ]:
print( u"\u20AC")

In [2]:
import torch
from fast_bert.data_cls import BertDataBunch 
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging

In [2]:
import locale; 
if locale.getpreferredencoding().upper() != 'UTF-8': 
    locale.setlocale(locale.LC_ALL, ('fr', 'utf-8'))

In [3]:
import sys
sys.getdefaultencoding()

'utf-8'

In [4]:
locale.getpreferredencoding()

'UTF-8'

In [5]:
locale.getlocale()

(None, None)

In [6]:
sys.getfilesystemencoding()

'utf-8'

In [7]:
import win_unicode_console
win_unicode_console.enable()

C:\Users\mat250\miniconda3\envs\fastcmb\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'utf-8', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)


In [8]:
sys.stdout.encoding

'UTF-8'

In [3]:
logging.basicConfig(
    handlers=[logging.FileHandler('logfile.log', 'w', 'utf-8')],
    format='%(levelname)s: %(message)s',
    datefmt='%m-%d %H:%M',
    level=logging.INFO #CRITICAL ERROR WARNING  INFO    DEBUG    NOTSET 
)
logger = logging.getLogger()


In [10]:
#device_cpu = torch.device("cpu")
device_cuda = torch.device("cuda")

In [11]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model/')
LABEL_PATH = Path('./labels/')

In [5]:
df = pd.read_csv('./data/labeled_data.csv',encoding = 'utf-8')

In [6]:
val_set = df.sample(frac=0.2, replace=False, random_state=42)
train_set = df.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv('./data/val_set.csv',encoding = 'utf-8')
train_set.to_csv('./data/train_set.csv',encoding = 'utf-8')

Nombre de commentaires dans le val_set: 65
Nombre de commentaires dans le train_set: 258


In [7]:
labels = df.columns[2:].to_list()
with open('./labels/labels.txt', 'w',encoding = 'utf-8') as f:
    for i in labels:
        f.write(i + "\n")

In [8]:
df_texts = pd.read_csv('./data/raw_data.csv',encoding = 'utf-8')
all_texts = df_texts['caption'].to_list()
all_texts = [str(t.encode('utf-8')) for t in all_texts]
print('Nombre de commentaires:', len(all_texts))

Nombre de commentaires: 3899


In [15]:
all_texts[:7]

["b'Ferme dimanche.... \\xf0\\x9f\\xa5\\xba\\xf0\\x9f\\xa5\\xba'",
 "b'Nul aucun acceuil pour leur client orange c est du chiffre et mensonge commercial vraiment de pire en pire et lorsque vous avez raisons sur leurs demarche mensong\\xc3\\xa8res et commercial ils vous disent appeler le sav qui ce trouve en tunisie et qui a 2400 km d ici ne savent pas vous reglet votre problemes parce que deja ils comprenne pas pourquoi c est nul en france sont incapables de gerer m as pannes Visit\\xc3\\xa9 en septembre'",
 'b"(Traduit par Google) Attendu dans une file d\'attente de 30 minutes que les employ\\xc3\\xa9s me disent qu\'ils ne vendent qu\'un seul forfait mobile trop cher de 20 Go de donn\\xc3\\xa9es pour 50 euros. Ils n\'offrent rien d\'autre. Un petit.  (Original) Waited in a 30 minute line for the employees to tell me that they only sell one overpriced mobile plan of 20GB of data for 50 euro. They do not offer anything else. A petty."',
 'b"Passage \\xc3\\xa0 la boutique Orange des Cham

In [17]:
type(all_texts[5])

str

### Création de LMDataBunch

In [9]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                data_dir=DATA_PATH,
                text_list=all_texts,
                tokenizer='camembert-base',
                batch_size_per_gpu=16, #was 16, even 8 won't do
                max_seq_length=512, #was 512
                multi_gpu=False,
                model_type='camembert-base',
                logger=logger)


### Création de LMLearner

In [12]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            is_fp16=False) #was true with gpu

C:\Users\mat250\miniconda3\envs\fastcmb\lib\site-packages\torch\cuda\__init__.py:132: UserWarning: 
    Found GPU0 GeForce GTX 760 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [14]:
print(torch.__config__.show())

PyTorch built with:
  - MSVC 191125547
  - Intel(R) Math Kernel Library Version 2019.0.4 Product Build 20190411 for Intel(R) 64 architecture applications
  - OpenMP 200203
  - CUDA Runtime 10.0
  - NVCC architecture flags: -gencode;arch=compute_35,code=sm_35;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_61,code=sm_61;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_50,code=compute_50
  - CuDNN 7.4.1
  - Magma 2.5.0
  - Build settings: BLAS=MKL, BUILD_NAMEDTENSOR=OFF, BUILD_TYPE=Release, CXX_FLAGS=/DWIN32 /D_WINDOWS /W3 /GR /EHsc /EHa -openmp /MP /bigobj, DISABLE_NUMA=1, PERF_WITH_AVX=1, PERF_WITH_AVX2=1, USE_CUDA=True, USE_EXCEPTION_PTR=1, USE_GFLAGS=OFF, USE_GLOG=OFF, USE_MKL=ON, USE_MKLDNN=OFF, USE_MPI=OFF, USE_NCCL=OFF, USE_NNPACK=OFF, USE_OPENMP=ON, 



In [13]:
lm_learner.fit(epochs=30, #was 30
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

RuntimeError: CUDA error: no kernel image is available for execution on the device

In [8]:
lm_learner.validate()

{'loss': 0.11761431582272053, 'perplexity': 1.1248102188110352}

In [10]:
lm_learner.save_model()

### Création de databunch pour la classification

In [11]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='review',
                          label_col=['cadre/atmosphère','probleme technique',"temps d'attente",'accueil/relation commerciale'],
                          batch_size_per_gpu=4,
                          max_seq_length=160,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')

### Création de Learner

In [12]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path('./finetuned_model')
WGTS_PATH = Path('model/model_out/pytorch_model.bin')

In [13]:
# issue fast-bert pos_weight <= downgrade to 1.9.1 solve the prob
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='model/model_out',
                        metrics=metrics,
                        device=device_cpu, #was device_cuda
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=False,#True when is cuda
                        logging_steps=50)

Some weights of the model checkpoint at model/model_out were not used when initializing CamembertForMultiLabelSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing CamembertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing CamembertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForMultiLabelSequenceClassification were not initialized from the model checkpoint at model/model_out and are newly initialized: ['classifier.dense.w

In [26]:
cl_learner.fit(epochs=30,
            lr=2e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

(1950, 0.10294121464738289)

In [27]:
cl_learner.validate()

{'loss': 0.2687712079948849,
 'fbeta': 0.9133089184761047,
 'roc_auc': 0.9533236354436785}

In [28]:
cl_learner.save_model()

### Prédictions

In [29]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [34]:
predictor.predict("Une vidéo publiée sur Twitter montre plusieurs camions parés de drapeaux pro-Trump encercler et ralentir le car de Joe Biden et Kamala Harris")

[('accueil/relation commerciale', 0.982572078704834),
 ("temps d'attente", 0.0077329580672085285),
 ('cadre/atmosphère', 0.00685591297224164),
 ('probleme technique', 0.006072716787457466)]